In [ ]:
!pip install gpt_index==0.4.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.4/969.4 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install langchain==0.0.90

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.8/281.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.10
    Uninstalling SQLAlchemy-2.0.10:
      Successfully uninstalled SQLAlchemy-2.0.10
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.188
    Uninstalling langchain-0.0.188:
      Successfully uninstalled langchain-0.0.188


In [ ]:
# Checking gpu 
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
# importing dependencies
from gpt_index import SimpleDirectoryReader, GPTListIndex, GPTSimpleVectorIndex, LLMPredictor, PromptHelper
from langchain import OpenAI
import sys
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = "YOUR-OPENAI-KEY"

In [ ]:
def createVectorIndex(path):
  max_input = 4096
  tokens = 200
  chunk_size = 600 #for LLM, we need to define chunk size
  max_chunk_overlap = 20
  
  #define prompt
  promptHelper = PromptHelper(max_input,tokens,max_chunk_overlap,chunk_size_limit=chunk_size)
  
  #define LLM — there could be many models we can use, but in this example, let’s go with OpenAI model
  llmPredictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="gpt-3.5-turbo",max_tokens=tokens))
  
  #load data — it will take all the .txtx files, if there are more than 1
  docs = SimpleDirectoryReader(path).load_data()

  #create vector index
  vectorIndex = GPTSimpleVectorIndex(documents=docs,llm_predictor=llmPredictor,prompt_helper=promptHelper)
  vectorIndex.save_to_disk('vectorIndex.json')
  return vectorIndex

In [ ]:
# converting text documents into vectors
vectorIndex = createVectorIndex("/content/documentation")

In [ ]:
# Creating helper function for querying the answer
def answerMe(vectorIndex):
  vIndex = GPTSimpleVectorIndex.load_from_disk(vectorIndex)
  while True:
    prompt = input("Please ask: ")
    if prompt == "exit":
      break
    response = vIndex.query(prompt, response_mode="compact")
    print(f"Response: {response} \n")


In [ ]:
# Querying from converted text file
answerMe("/content/vectorIndex.json")

Please ask: Hi
Response: 
Hi! How can I help you? 

Please ask: dataset details
Response: 
The Plant Diseases Dataset contains 38,000 images of plants affected by various diseases. It includes 38 different classes or categories, such as Apple Scab, Apple Black Rot, Apple Cedar Rust, Apple Healthy, Blueberry Healthy, Cherry Healthy, Cherry Powdery Mildew, Corn Gray Leaf Spot, Corn Common Rust, Corn Northern Leaf Blight, Corn Healthy, Grape Black Rot, and Grape Esca (Black Measles). 

Please ask: images categories
Response: 
The images categories are: Apple Healthy, Cherry Healthy, Corn Healthy, Grape Healthy, Raspberry Healthy, Esca (Black Measles), Grape Leaf Blight (Isariopsis Leaf Spot), Orange Huanglongbing (Citrus Greening), Peach Bacterial Spot, Peach Healthy, Bell Pepper Bacterial Spot, Bell Pepper Healthy, Potato Early Blight, Potato Late Blight, Potato Healthy, Soybean Healthy, Squash Powdery Mildew, Strawberry Leaf Scorch, Strawberry Healthy, Tomato Bacterial Spot, Tomato Earl